# <<<>>>안에 <=>이게 없을 경우만 골라서 해당 행 추출

In [1]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [2]:
# 해당 경로의 파일리스트 추출
def get_filelist(input_repo):
    file_list = []
    for (dirpath, dirnames, filenames) in walk(input_repo):
        file_list.extend(filenames)
        break

    return file_list

In [3]:
# 해당 경로의 csv파일의 경로+파일명 리스트 추출
def get_csvlist(input_repo, file_list):
    csv_list = []
    for i in range(len(file_list)):
        filename, file_extension = os.path.splitext(f'{input_repo}/{file_list[i]}')

        if file_extension == '.csv':
            csv_list.append(filename+file_extension)

    return csv_list

In [4]:
def concat_csvfile(csv_list):
    COLUMNS = ['lineNumber', 'originalText', 'transcription']
    result = pd.concat([pd.read_csv(f) for f in csv_list])    # 라인넘버 기준 안합친파일 합치기
    sorted_result_not_flat = result.sort_values(by=['lineNumber'])    # 작업물 sort by lineNumber
    sorted_result_not_flat.to_csv(f'{output_repo}/temp.csv', index=False, encoding='utf-8-sig')
    sorted_result = pd.read_csv(f'{output_repo}/temp.csv')
    os.remove(f'{output_repo}/temp.csv')
    
    return sorted_result

In [5]:
# 전체 문장으로부터 태그 제거
def remove_tag(sentence):
    result = sentence.replace('<<<', '').replace('>>>', '')
        
    return result

In [6]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    try:
        return result.group()
    except:
        return 'no word'

In [7]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 법률어휘오류_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/법률어휘오류_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [8]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/concat'
ref_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/accum_concat/법률어휘 살리기 최종'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/accum_concat/법률어휘 남은거만 뽑아냄'

ref_filename = 'total_with_lineNumber.csv'

In [9]:
file_list = get_filelist(input_repo)
csv_list = get_csvlist(input_repo, file_list)
sorted_result = concat_csvfile(csv_list)
ref_word = pd.read_csv(f'{ref_repo}/{ref_filename}')

In [10]:
ref_word

,lineNumber,word,org,tagged
0,1,호환,"원심판결 이유에 의하면, 원심은 이 사건 행위가 공정거래법 제3조의2 제1항 제3호...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이..."
1,2,호환,비록 양 발명의 구성에 상이점이 있어도 그 기술분야에서 통상의 지식을 가진 자가 보...,비록 양 발명의 구성에 <<<서로 다름<=상이>점이 있어도>>> 그 기술분야에서 <...
2,3,호환,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 호환성이 있어 의장법...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문..."
3,4,호환,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 호환컴퓨터에 대표적으...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...
4,5,호환,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ..."
...,...,...,...,...
75995,75996,종용,1. 기록에 비추어 피고가 시외버스운송사업에 관한 권한을 자동차운송사업법 제62조 ...,1. 기록에 비추어 <<<민사소송에서 소송을 당한 사람<=피고>가 시외버스운송사업에...
75996,75997,종용,고소인과 피고인이 같이 거주하던 아파트의 열쇠를 넘겨받고 아울러 피고인 명의로 등기...,고소인과 <<<형사소송에서 검사에의해 기소당한 사람<=피고인>이 같이>>> 거주하던...
75997,75998,종용,"원심은 거시증거를 종합하여 ****.**.**원고 회사의 무역부장인 소외 ***, ...",<<<직전의 재판<=원심>은 거시증거를>>> 종합하여 ****.**.**원고 회사의...
75998,75999,종용,원고는 자신이 순환명령휴직 대상자에 선정될 것을 예상하고 그와 같은 경우 휴직기간 ...,<<<민사소송을 일으킨 사람<=원고>는 자신이>>> 순환명령휴직 대상자에 <<<여러...


In [11]:
sorted_result

,lineNumber,originalText,transcription,name
0,1,"<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이...",NaN
1,2,비록 양 발명의 구성에 <<<서로 다름<=상이>점이 있어도>>> 그 기술분야에서 <...,비록 양 발명의 구성에 <<<서로 다른<=상이> 점이 있어도>>> 그 기술분야에서 ...,NaN
2,3,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문...",NaN
3,4,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...,NaN
4,5,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...",NaN
...,...,...,...,...
52432,75991,이에 피해자의 동생은 <<<형사소송에서 검사에의해 기소당한 사람<=피고인>의 뜻이>...,이에 피해자의 동생은 <<<형사소송에서 검사에의해 기소당한 사람<=피고인>의 뜻이>...,NaN
52433,75992,"[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자...","[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자...",NaN
52434,75993,［2］ 간통죄의 고소인과 그 배우자인 <<<민사소송에서 소송을 당한 사람<=피고>소...,［2］ 간통죄의 고소인과 그 배우자인 <<<민사소송에서 소송을 당한 사람<=피고>소...,NaN
52435,75994,"<<<직전의 재판<=원심>은, 민사소송을 일으킨 사람<=원고>가>>> <<<민사소송...","<<<직전의 재판<=원심>은, 민사소송을 일으킨 사람<=원고>이>>> <<<민사소송...",NaN


In [12]:
# 법률어휘 word를 no_word_dataframe에 추가
no_word_dataframe = pd.DataFrame(columns=['word', 'lineNumber', 'originalText', 'transcription'])
for i in tnrange(len(sorted_result)):
    if draw_word(remove_tag(str(sorted_result['transcription'][i]))) == 'no word':
        ref_lineNumber = sorted_result.loc[i]['lineNumber']
        word = ref_word[ref_word['lineNumber']=sorted_result['lineNumber'][i]
        no_word_dataframe = no_word_dataframe.append({'lineNumber':sorted_result['lineNumber'][i], 'originalText':sorted_result['originalText'][i], 'transcription':sorted_result['transcription'][i]}, ignore_index=True)
    else:
        pass
# 'word':list(ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][i]]['word'])[0], 

  0%|          | 0/52437 [00:00<?, ?it/s]

In [31]:
list(ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][0]]['word'])[0]

'탈환'

In [34]:
ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][2]]

,lineNumber,word,org,tagged
602,603,임시직,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부


In [20]:
list(ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][0]]['word'])[0]

'탈환'

In [24]:
list(ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][0]]['word'])[1]

IndexError: list index out of range

In [13]:
no_word_dataframe

,lineNumber,originalText,transcription
0,396,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...
1,439,대의민주주의에서 후보자나 정당 등에 관한 정치적 정보 및 의견을 자유롭게 발표하고 ...,대의민주주의에서 후보자나 정당 등에 관한 정치적 정보 및 의견을 자유롭게 발표하고 ...
2,603,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부
3,619,나. 공무원연금법의 적용대상에서 제외된 지방 잡급직원등 임시직 공무원이,나. 공무원연금법의 적용대상에서 제외된 지방 잡급직원등 임시직 공무원이
4,622,국가공무원법 제46조 제4항)은 임시직공무원에 대한,국가공무원법 제46조 제4항)은 임시직공무원에 대한
...,...,...,...
2219,74593,형사소송에서도 판결 확정에 따른 실체적 확정력을 인정함으로써 법적 안정성을 도모하여...,형사소송에서도 판결 확정에 따른 실체적 확정력을 인정함으로써 법적 안정성을 도모하여...
2220,74606,[1] 오늘날 자동차가 <<<갑작스럽게 증가함<=급증>하고 자동차운전면허도>>> 대...,[1] 오늘날 자동차가 <<<급증하고 자동차운전면허도>>> 대량으로 <<<발급 되어...
2221,74608,이는 <<<갑작스럽게 증가함<=급증>하는 스팸형>>> 메일이나 문자메시지 등을 통한...,이는 <<<급증하는 스팸형>>> 메일이나 문자메시지 등을 통한 무차별적인 광고성 정...
2222,74625,객관적으로 타당한 법적 근거와 합리성이 없는지 <<<맞거나 틀림<=여부>는 대법원판...,객관적으로 타당한 법적 근거와 합리성이 없는지 <<<여부는 대법원판결이>>> 선고되...


In [31]:
# no_word_dataframe에서 lineNumber갖고와서 ref_word의 word 추출
no_word_dataframe

,lineNumber,originalText,transcription
0,396,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...
1,439,대의민주주의에서 후보자나 정당 등에 관한 정치적 정보 및 의견을 자유롭게 발표하고 ...,대의민주주의에서 후보자나 정당 등에 관한 정치적 정보 및 의견을 자유롭게 발표하고 ...
2,603,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부
3,619,나. 공무원연금법의 적용대상에서 제외된 지방 잡급직원등 임시직 공무원이,나. 공무원연금법의 적용대상에서 제외된 지방 잡급직원등 임시직 공무원이
4,622,국가공무원법 제46조 제4항)은 임시직공무원에 대한,국가공무원법 제46조 제4항)은 임시직공무원에 대한
...,...,...,...
2219,74593,형사소송에서도 판결 확정에 따른 실체적 확정력을 인정함으로써 법적 안정성을 도모하여...,형사소송에서도 판결 확정에 따른 실체적 확정력을 인정함으로써 법적 안정성을 도모하여...
2220,74606,[1] 오늘날 자동차가 <<<갑작스럽게 증가함<=급증>하고 자동차운전면허도>>> 대...,[1] 오늘날 자동차가 <<<급증하고 자동차운전면허도>>> 대량으로 <<<발급 되어...
2221,74608,이는 <<<갑작스럽게 증가함<=급증>하는 스팸형>>> 메일이나 문자메시지 등을 통한...,이는 <<<급증하는 스팸형>>> 메일이나 문자메시지 등을 통한 무차별적인 광고성 정...
2222,74625,객관적으로 타당한 법적 근거와 합리성이 없는지 <<<맞거나 틀림<=여부>는 대법원판...,객관적으로 타당한 법적 근거와 합리성이 없는지 <<<여부는 대법원판결이>>> 선고되...


In [54]:
ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][0]]

,lineNumber,word,org,tagged
395,396,탈환,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...


In [59]:
word_list = []
for i in range(len(no_word_dataframe)):
    word_list.append(list(ref_word[ref_word['lineNumber']==no_word_dataframe['lineNumber'][i]]['word'])[0])

In [63]:
no_word_dataframe['word'] = word_list
no_word_dataframe

,lineNumber,originalText,transcription,word
0,396,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...,(1) **** *****연대 소속 일부 군인들이 ****.**.**제주 4·3 사...,탈환
1,439,대의민주주의에서 후보자나 정당 등에 관한 정치적 정보 및 의견을 자유롭게 발표하고 ...,대의민주주의에서 후보자나 정당 등에 관한 정치적 정보 및 의견을 자유롭게 발표하고 ...,대의
2,603,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부,임시직원이 퇴직 1개월후 정식직원이 된 경우 고용관계 계속여부,임시직
3,619,나. 공무원연금법의 적용대상에서 제외된 지방 잡급직원등 임시직 공무원이,나. 공무원연금법의 적용대상에서 제외된 지방 잡급직원등 임시직 공무원이,임시직
4,622,국가공무원법 제46조 제4항)은 임시직공무원에 대한,국가공무원법 제46조 제4항)은 임시직공무원에 대한,임시직
...,...,...,...,...
2219,74593,형사소송에서도 판결 확정에 따른 실체적 확정력을 인정함으로써 법적 안정성을 도모하여...,형사소송에서도 판결 확정에 따른 실체적 확정력을 인정함으로써 법적 안정성을 도모하여...,법적 안정성
2220,74606,[1] 오늘날 자동차가 <<<갑작스럽게 증가함<=급증>하고 자동차운전면허도>>> 대...,[1] 오늘날 자동차가 <<<급증하고 자동차운전면허도>>> 대량으로 <<<발급 되어...,급증
2221,74608,이는 <<<갑작스럽게 증가함<=급증>하는 스팸형>>> 메일이나 문자메시지 등을 통한...,이는 <<<급증하는 스팸형>>> 메일이나 문자메시지 등을 통한 무차별적인 광고성 정...,급증
2222,74625,객관적으로 타당한 법적 근거와 합리성이 없는지 <<<맞거나 틀림<=여부>는 대법원판...,객관적으로 타당한 법적 근거와 합리성이 없는지 <<<여부는 대법원판결이>>> 선고되...,선후


In [12]:
save_as_csv(input_repo, output_repo, no_word_dataframe)

/Users/jaytsol/Desktop/Type23 (2)/누적/accum_concat/법률어휘 남은거만 뽑아냄의 경로에 법률어휘오류_211110_111.csv의 이름으로 파일 생성이 완료되었습니다.
